In [1]:
import rospy
from geometry_msgs.msg import Twist, Point, Quaternion
from math import radians, copysign, sqrt, pow, pi, atan2
import time
from threading import Thread
from nav_msgs.msg import Odometry

In [2]:
import numpy
import math

_AXES2TUPLE = {
    'sxyz': (0, 0, 0, 0), 'sxyx': (0, 0, 1, 0), 'sxzy': (0, 1, 0, 0),
    'sxzx': (0, 1, 1, 0), 'syzx': (1, 0, 0, 0), 'syzy': (1, 0, 1, 0),
    'syxz': (1, 1, 0, 0), 'syxy': (1, 1, 1, 0), 'szxy': (2, 0, 0, 0),
    'szxz': (2, 0, 1, 0), 'szyx': (2, 1, 0, 0), 'szyz': (2, 1, 1, 0),
    'rzyx': (0, 0, 0, 1), 'rxyx': (0, 0, 1, 1), 'ryzx': (0, 1, 0, 1),
    'rxzx': (0, 1, 1, 1), 'rxzy': (1, 0, 0, 1), 'ryzy': (1, 0, 1, 1),
    'rzxy': (1, 1, 0, 1), 'ryxy': (1, 1, 1, 1), 'ryxz': (2, 0, 0, 1),
    'rzxz': (2, 0, 1, 1), 'rxyz': (2, 1, 0, 1), 'rzyz': (2, 1, 1, 1)}

_TUPLE2AXES = dict((v, k) for k, v in _AXES2TUPLE.items())

# epsilon for testing whether a number is close to zero
_EPS = numpy.finfo(float).eps * 4.0

# axis sequences for Euler angles
_NEXT_AXIS = [1, 2, 0, 1]

def euler_from_quaternion(quaternion, axes='sxyz'):
    """Return Euler angles from quaternion for specified axis sequence.
    >>> angles = euler_from_quaternion([0.06146124, 0, 0, 0.99810947])
    >>> numpy.allclose(angles, [0.123, 0, 0])
    True
    """
    return euler_from_matrix(quaternion_matrix(quaternion), axes)[2]

def euler_from_matrix(matrix, axes='sxyz'):
    """Return Euler angles from rotation matrix for specified axis sequence.
    axes : One of 24 axis sequences as string or encoded tuple
    Note that many Euler angle triplets can describe one matrix.
    >>> R0 = euler_matrix(1, 2, 3, 'syxz')
    >>> al, be, ga = euler_from_matrix(R0, 'syxz')
    >>> R1 = euler_matrix(al, be, ga, 'syxz')
    >>> numpy.allclose(R0, R1)
    True
    >>> angles = (4.0*math.pi) * (numpy.random.random(3) - 0.5)
    >>> for axes in _AXES2TUPLE.keys():
    ...    R0 = euler_matrix(axes=axes, *angles)
    ...    R1 = euler_matrix(axes=axes, *euler_from_matrix(R0, axes))
    ...    if not numpy.allclose(R0, R1): print axes, "failed"
    """
    try:
        firstaxis, parity, repetition, frame = _AXES2TUPLE[axes.lower()]
    except (AttributeError, KeyError):
        _ = _TUPLE2AXES[axes]
        firstaxis, parity, repetition, frame = axes

    i = firstaxis
    j = _NEXT_AXIS[i+parity]
    k = _NEXT_AXIS[i-parity+1]

    M = numpy.array(matrix, dtype=numpy.float64, copy=False)[:3, :3]
    if repetition:
        sy = math.sqrt(M[i, j]*M[i, j] + M[i, k]*M[i, k])
        if sy > _EPS:
            ax = math.atan2( M[i, j],  M[i, k])
            ay = math.atan2( sy,       M[i, i])
            az = math.atan2( M[j, i], -M[k, i])
        else:
            ax = math.atan2(-M[j, k],  M[j, j])
            ay = math.atan2( sy,       M[i, i])
            az = 0.0
    else:
        cy = math.sqrt(M[i, i]*M[i, i] + M[j, i]*M[j, i])
        if cy > _EPS:
            ax = math.atan2( M[k, j],  M[k, k])
            ay = math.atan2(-M[k, i],  cy)
            az = math.atan2( M[j, i],  M[i, i])
        else:
            ax = math.atan2(-M[j, k],  M[j, j])
            ay = math.atan2(-M[k, i],  cy)
            az = 0.0

    if parity:
        ax, ay, az = -ax, -ay, -az
    if frame:
        ax, az = az, ax
    return ax, ay, az

def quaternion_matrix(quaternion):
    """Return homogeneous rotation matrix from quaternion.
    >>> R = quaternion_matrix([0.06146124, 0, 0, 0.99810947])
    >>> numpy.allclose(R, rotation_matrix(0.123, (1, 0, 0)))
    True
    """
    q = numpy.array(quaternion[:4], dtype=numpy.float64, copy=True)
    nq = numpy.dot(q, q)
    if nq < _EPS:
        return numpy.identity(4)
    q *= math.sqrt(2.0 / nq)
    q = numpy.outer(q, q)
    return numpy.array((
        (1.0-q[1, 1]-q[2, 2],     q[0, 1]-q[2, 3],     q[0, 2]+q[1, 3], 0.0),
        (    q[0, 1]+q[2, 3], 1.0-q[0, 0]-q[2, 2],     q[1, 2]-q[0, 3], 0.0),
        (    q[0, 2]-q[1, 3],     q[1, 2]+q[0, 3], 1.0-q[0, 0]-q[1, 1], 0.0),
        (                0.0,                 0.0,                 0.0, 1.0)
        ), dtype=numpy.float64)


In [3]:
def callback(data):
    global dx,dy,angle
    dx = data.pose.pose.position.x
    dy = data.pose.pose.position.y
    
    quat = [data.pose.pose.orientation.x,
            data.pose.pose.orientation.y,
            data.pose.pose.orientation.z,
            data.pose.pose.orientation.w]
    
    angle = math.degrees(euler_from_quaternion(quat))

In [5]:
def measurementThread():
    running_thread = True
    
    # Create a suscribe to Odometry node using callback function   
    odomSuscriber = rospy.Subscriber('/odom', Odometry, callback)
    
    rate = rospy.Rate(1)
    rate.sleep()
    rospy.spin()

In [6]:
def calculateDistance(x0,y0,x1,y1):
    return math.sqrt((x0-x1)**2+(y0-y1)**2)

In [7]:
def Move(distance=1,speed=0.2):
    direction = -1 if speed < 0 else 1
    global dx,dy,angle
    global real_angle, real_x, real_y
    global moved_distance, movements
    
    move = Twist()
    move.linear.x=speed
    r = rospy.Rate(2)
    
    init_x = dx
    init_y = dy
    
    while calculateDistance(init_x,init_y,dx,dy) < distance*0.95:
        pub.publish(move)
        time.sleep(0.8)
    moved_distance = calculateDistance(init_x,init_y,dx,dy)*direction
    movements.append(moved_distance)
    print('/nmoved: ', moved_distance)
    #print(f'shitty coordinates: angle={angle}, y={dx}, x={dy}')

In [8]:
def Rotate(degrees=10, direction='left'):
    global dx,dy,angle
    global angle
    global real_angle
    
    # Angle symbol
    direct = 1 if direction=='left' else -1
    
    # Create msg to communicate with ROS
    rotate = Twist()
    
    # Define speed and direction of rotation
    rotate.angular.z = direct*0.175
    r = rospy.Rate(2)

    # Rotate and calculate how much it rotated
    init_angle = angle
    while abs(init_angle-angle) < degrees:#*0.9:
        pub.publish(rotate)
        time.sleep(0.8)
        
    # Update real_angle
    rotated=direct*abs(init_angle-angle)
    real_angle-=rotated
    
    # Make sure real_angle is between 0-360
    if real_angle/360>1:
        real_angle-=360
    elif real_angle<0:
        real_angle+=360
    
    print('rotated: ', rotated)
    rotations.append(rotated)
    #print(f'shitty coordinates: angle={angle}, y={dx}, x={dy}')

In [9]:
def update_coordinates():
    
    time.sleep(0.5)
    
    global moved_distance
    global real_angle, real_x, real_y
    
    if real_angle == 0: # Moved Forward ==> Y++
        real_y+=moved_distance
        
    elif real_angle == 90: # Moved Right ==> X++
        real_x+=moved_distance
        
    elif real_angle == 180: # Moved Backward ==> Y--
        real_y-=moved_distance
        
    elif real_angle == 270: # Moved Left ==> X--
        real_x-=moved_distance
        
    else:  # ==> Calculate X,Y components 
        delta_x = moved_distance*math.sin(math.radians(real_angle))
        delta_y = moved_distance*math.cos(math.radians(real_angle))
               
        real_x += delta_x
        real_y += delta_y